[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/racousin/rag_attack/blob/main/rag_attack.ipynb)

## Setup Initial pour Google Colab

In [ ]:
# Clone the repository
!git clone https://github.com/racousin/rag_attack.git
%cd rag_attack

In [ ]:
# Run installation script
!bash install_colab.sh

In [ ]:
# Install the rag_attack package
!pip install -e .

## Configuration

⚠️ **Important**: Remplacez les valeurs vides ci-dessous avec vos propres credentials Azure

In [ ]:
# Configuration - DEMANDEZ VOS CREDENTIALS
config = {
    'search_endpoint': '',  # Ex: 'https://your-search.search.windows.net'
    'search_key': '',  # Votre clé Azure Search
    'sql_server': '',  # Ex: 'your-server.database.windows.net'
    'sql_database': '',  # Nom de votre base de données
    'sql_username': '',  # Username SQL
    'sql_password': '',  # Password SQL
    'api_base_url': '',  # Ex: 'https://your-api.azurewebsites.net/api'
    'openai_endpoint': '',  # Ex: 'https://your-region.api.cognitive.microsoft.com/'
    'openai_key': '',  # Votre clé OpenAI/Azure OpenAI
    'chat_deployment': ''  # Ex: 'gpt-4' ou 'gpt-35-turbo'
}

# Validate configuration using the rag_attack package
from rag_attack import validate_config, test_connection

# Test that the package is loaded
print(test_connection())

# Validate configuration
is_valid, message = validate_config(config)
if is_valid:
    print("✅", message)
else:
    print("❌", message)
    print("\nVeuillez remplir tous les champs de configuration avant de continuer.")

# RAG Agentique avec LangGraph - VéloCorp

Ce notebook démontre un système RAG agentique utilisant LangGraph pour orchestrer l'accès à 3 sources de données :
- 🔍 **Azure Search** - Documents, FAQ, manuels
- 🏢 **Base de données** - Données métier (commandes, clients, produits)
- 📊 **API CRM** - Commerciaux, prospects, opportunités

In [3]:
# Nécessaire d'effecture ceci avant d'importer Azure Search sinon Azure utilise
# sa nomenclature par défaut
import os
os.environ["AZURESEARCH_FIELDS_CONTENT_VECTOR"] = "embedding"

In [4]:
# Imports standards :
import functools
import json
import logging
import os
import re
import subprocess
import time
from datetime import datetime
from typing import Annotated, Dict, Optional, TypedDict

# Imports Azure
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient

try:
    from azure.search.documents.models import VectorizedQuery
except ImportError:
    VectorizedQuery = None

from azure.identity import (
    DeviceCodeCredential,
    EnvironmentCredential,
    ChainedTokenCredential,
)
from azure.keyvault.secrets import SecretClient
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.search import SearchManagementClient
from azure.mgmt.sql import SqlManagementClient
from azure.mgmt.web import WebSiteManagementClient
from azure.mgmt.cognitiveservices import CognitiveServicesManagementClient

# Imports third-party
import pyodbc
import requests
from dotenv import load_dotenv
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import AzureSearch
from langchain_community.vectorstores import FAISS
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langchain_openai import (
    ChatOpenAI,
    AzureChatOpenAI,
    AzureOpenAIEmbeddings,
)
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from typing import List, Optional, Dict, Any
from google.colab import files

ModuleNotFoundError: No module named 'google'

In [ ]:
# Ce bloc permet d’installer et de configurer le driver ODBC 18 de Microsoft pour se connecter à SQL Server.

# 1. Mise à jour de la liste des paquets et installation des outils ODBC
!apt-get update
!apt-get install -y unixodbc unixodbc-dev

# 2. Installation du driver MS ODBC 18
!curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
!apt-get install -y curl gnupg
!curl https://packages.microsoft.com/config/ubuntu/22.04/prod.list | tee /etc/apt/sources.list.d/mssql-release.list
!apt-get update
!ACCEPT_EULA=Y apt-get install -y msodbcsql18

# 3. Vérification
!odbcinst -j
!odbcinst -q -d
!odbcinst -q -d -n "ODBC Driver 18 for SQL Server"

# 1/ RAG simple

Etape 1 : charger les manuels utilisateur des vélos

In [ ]:
# Récupérer le fichier local
uploaded = files.upload()

# Récupérer le nom du fichier uploadé dans colab
filename = next(iter(uploaded))

In [ ]:
def manuals_json_loader(json_path: str) -> List[Dict]:
    """
    Charge uniquement les documents de type 'manuel_technique' à partir d'un fichier JSON unifié.

    Args:
        json_path (str): Chemin vers le fichier JSON fusionné contenant tous les documents.

    Returns:
        list of dict: Liste de dictionnaires contenant l'id, les métadonnées et le texte des manuels techniques.
    """
    # Ouvre le fichier JSON et charge toutes les données en mémoire
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    manuals = []  # Liste qui stockera tous les manuels techniques extraits

    # Parcourt chaque entrée du JSON unifié
    for key, item in data.items():
        meta = item.get("metadata", {})
        # On vérifie que le document est bien un manuel technique
        if meta and meta.get("type", "").lower() == "manuel_technique":
            manuals.append({
                "id": key,                # identifiant du document (ex: "manuel_e_sport")
                "metadata": meta,         # toutes les métadonnées associées
                "text": item.get("text", "")  # le texte intégral du manuel
            })
    return manuals

manuals = manuals_json_loader(filename)
manuals[0]

In [ ]:
def split_by_section(text: str) -> List[str]:
    """
    Découpe un texte en sections à partir des titres de section en majuscules.
    Chaque chunk commence par un titre, suivi de son contenu.

    Args:
        text (str): Le texte du manuel complet.
    Returns:
        list of str: Liste des sections trouvées dans le texte.
    """
    # Regex pour trouver les titres de section (grandes majuscules sur une ligne)
    section_pattern = r"\n([A-ZÉÈÀÇ\- ]{8,})\n"
    # On split sur chaque titre détecté, ce qui donne :
    # [introduction, spécification techniques, montage, résolution de probèmes, ...]
    sections = re.split(section_pattern, text)
    chunks = []
    # sections[0] = introduction éventuelle (avant le premier titre)
    intro = sections[0].strip()
    if intro:
        chunks.append(intro)  # On garde l'intro si elle existe
    # On parcourt les couples (titre, contenu)
    for i in range(1, len(sections), 2): # pas de 2 pour sauter les contenus
        title = sections[i].strip()
        content = sections[i+1].strip() if i+1 < len(sections) else ""
        if content:
            # On recompose "titre + contenu" pour chaque chunk
            chunk = f"{title}\n{content}"
            chunks.append(chunk)
    return chunks

def make_chunk_with_context(doc: Dict[str, Any], section_chunk: str) -> str:
    """
    Ajoute un en-tête de contexte (modèle, type, ligne de produit) au début de chaque chunk.
    Cela permet à chaque chunk de garder l'identité du produit même si le texte ne le mentionne pas.

    Args:
        doc (dict): Dictionnaire contenant 'metadata' sur le produit.
        section_chunk (str): Le texte du chunk (section).
    Returns:
        str: Le chunk enrichi du contexte produit.
    """
    header = f"Modèle : {doc['metadata'].get('model', '')}\n"
    header += f"Type : {doc['metadata'].get('type', '')}\n"
    header += f"Ligne de produit : {doc['metadata'].get('product_line', '')}\n"
    return header + section_chunk

from langchain.text_splitter import RecursiveCharacterTextSplitter

def chunking(
        manuals: List[Dict[str, Any]],
        chunk_size: int = 350,
        chunk_overlap: int = 50
    ) -> List[Dict[str, Any]]:
    """
    Découpe chaque manuel technique en chunks de taille fixe, avec overlap,
    en gardant le contexte du modèle dans chaque chunk.
    """
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ".", " "]
    )
    all_chunks = []
    for doc in manuals:
        # Nettoyage des séparateurs visuels
        clean_text = re.sub(r"(^[-\s]{3,}$\n?)+", "", doc["text"], flags=re.MULTILINE)
        # Découpage global du manuel en chunks overlapping
        chunks = splitter.split_text(clean_text)
        for i, chunk in enumerate(chunks):
            chunk_text = make_chunk_with_context(doc, chunk)
            all_chunks.append({
                "doc_id": doc["id"],
                "chunk_id": f"{doc['id']}_chunk_{i}",
                "metadata": doc["metadata"],
                "text": chunk_text
            })
    return all_chunks

Utilisation du code de chunking et visualisation des résultats :

In [ ]:
all_chunks = chunking(manuals)

# Affichage d'exemple
for chunk in all_chunks[:2]:
    print("="*100)
    print(chunk["text"])

### Désormais, nous allons vectoriser (embedding) nos chunks et les stocker pour créer un Vector Store.
Un Vector Store est une base de données spécialisée qui stocke des représentations numériques (embeddings) de textes ou documents pour permettre une recherche sémantique rapide.
Elle permet de retrouver efficacement les passages les plus pertinents par similarité de sens, même si les mots exacts ne sont pas présents dans la requête de l’utilisateur.

FAISS est une bibliothèque open source permettant la recherche ultra-rapide de similarité entre vecteurs dans de grands volumes de données.
On l’utilise pour retrouver efficacement les passages ou documents les plus pertinents à partir d’une requête, même dans des bases contenant des millions de textes ou d’images.
Parmi les alternatives, on trouve ChromaDB, Qdrant, Weaviate, Milvus, ou encore Annoy et ScaNN.


In [ ]:
def create_faiss_vectorstore(
        embedded_chunks: List[Dict[str, Any]],
        model_name: str = 'sentence-transformers/all-MiniLM-L6-v2'
    ) -> FAISS:
    """
    Prend la liste de chunks embeddés, construit un vector store FAISS compatible LangChain/LangGraph.
    Retourne le vector store prêt à l'emploi.
    """
    # 1. Création des objets Document avec contenu et métadonnées
    docs = [
        Document(
            page_content=chunk["text"],
            metadata=chunk["metadata"]
        )
        for chunk in embedded_chunks
    ]

    # 2. Création de la fonction d'embedding (wrapper LangChain)
    embedding_fn = HuggingFaceEmbeddings(model_name=model_name)

    # 3. Création du vector store FAISS
    vectorstore = FAISS.from_documents(docs, embedding_fn)

    return vectorstore

In [ ]:
# Utilisation de la fonction
vectorstore = create_faiss_vectorstore(all_chunks)
print(f"Vector store FAISS créé avec {vectorstore.index.ntotal} chunks.")

In [ ]:
# Stockage du vectorstore
# Chemin de sauvegarde :
persist_directory = "/content/data/faiss_vectorstore"
os.makedirs(persist_directory, exist_ok=True)
print(f"Dossier créé à : {persist_directory}")

# Sauvegarde du vector store
vectorstore.save_local(persist_directory)
print(f"Vector store FAISS sauvegardé à : {persist_directory}")

Nous pouvons dorénavant utiliser notre RAG :

In [ ]:
# L'utilisateur saisit sa requête en langage naturel
query = "Quel est le poids du vélo Sport-Elite ?"

Inutile de générer manuellement l'embedding de la requête !
La méthode 'similarity_search' va automatiquement :
   - Passer la requête dans le même modèle d'embedding utilisé pour l'index
   - Comparer le vecteur résultant à tous les embeddings des chunks
   - Retourner les k passages les plus proches (selon la similarité cosinus)
C'est donc un appel "texte en entrée" ➔ "texte(s) le(s) plus pertinent(s) en sortie".


In [ ]:
results = vectorstore.similarity_search(query, k=3)

for i, doc in enumerate(results):
    print(f"\n--- Résultat #{i+1} ---")
    print("Texte du chunk :")
    print(doc.page_content)
    print("Métadonnées :")
    print(doc.metadata)

1/ Instanciation du LLM

In [ ]:
llm = AzureChatOpenAI(
    azure_endpoint  = config["openai_endpoint"],
    api_key         = config["openai_key"],
    deployment_name = config["chat_deployment"],
    api_version     = "2024-02-15-preview"
)

2/ Exemple de prompt bien formulé pour que le LLM fasse de la synthèse RAG

In [ ]:
prompt = """
Tu es un assistant expert, spécialisé dans la documentation technique des vélos.

Réponds de façon précise et concise à la question utilisateur suivante,
en t’appuyant STRICTEMENT et EXCLUSIVEMENT sur les extraits ci-dessous (issus de la documentation technique officielle) :
\"\"\"
{context}
\"\"\"

INSTRUCTIONS :
- Ne donne aucune information qui ne provient pas explicitement de ces extraits.
- Si la réponse exacte (par exemple, une valeur numérique, un modèle, un tableau) est présente, cite-la textuellement et indique d'où elle provient.
- Si plusieurs passages sont nécessaires pour une réponse complète, combine-les de façon claire.
- Si l’information demandée n’apparaît pas dans les extraits, dis simplement : « L’information ne figure pas dans la documentation fournie. »
- Reste factuel, sans extrapolation ni interprétation.

Question :
{query}
"""

3/ Récupérer les chunks les plus pertinents pour la requête

In [ ]:
query = "Quel est le poids vélo Urbain-Classic ?"
results = vectorstore.similarity_search(query, k=2)  # on prend les 2 passages les plus pertinents
context = "\n\n".join([doc.page_content for doc in results])

4/ Construire le prompt à envoyer au LLM, en insérant le contexte récupéré et envoyer au LLM

In [ ]:
response = llm.invoke(prompt.format(context=context, query=query))
print("Réponse du LLM :\n", response.content)

6/ comparaison avec une réponse générée sans RAG

In [ ]:
print(llm.invoke(query).content)

7/ Test pour une question dont les éléments de réponse sont absents de la documentation

In [ ]:
query = "De quel matériau est fabriqué le vélo urbain-confort ?"
results = vectorstore.similarity_search(query, k=2)
context = "\n\n".join([doc.page_content for doc in results])
response = llm.invoke(prompt.format(context=context, query=query))
print("Réponse du LLM :\n", response.content)

## 2/ Construction et utilisation d'un RAG encapsulé dans une classe avec Azure
#### Ce RAG suit la même logique que précédemment, mais en utilisant Azure. Il est encapsulé dans une classe afin de pouvoir le réutiliser comme Tool pour notre RAG agentique.

#### Récuperation des outputs terraform

#### Instanciation des LLMs via Azure

In [ ]:
llm = AzureChatOpenAI(
    azure_endpoint  = config["openai_endpoint"],
    api_key         = config["openai_key"],
    deployment_name = config["chat_deployment"],
    api_version     = "2024-02-15-preview"
)


#### Création d'une classe pour récupérer uniquement les manuels utilisateurs

In [ ]:
# -----------------------------------------------------------------------------
# AzureSearchFilteredRetriever
# -----------------------------------------------------------------------------
# Petit utilitaire pour interroger directement un index Azure Cognitive Search
# en mode "vector search" + filtre OData (ex: ne récupérer que les documents
# dont le champ 'type' vaut 'manuel').
#
# Pourquoi ce helper ?
# - Le wrapper LangChain AzureSearch ne propage pas toujours bien les filtres.
# - Ici on utilise le SDK officiel azure-search-documents pour garder le contrôle.
#
# Paramètres attendus :
#   endpoint   : URL du service Azure Search (ex: "https://xxx.search.windows.net")
#   index_name : nom de l'index (ex: "documents")
#   key        : clé admin ou query key autorisée
#   embed_fn   : fonction Python qui transforme un texte en vecteur (liste de floats)
#
# Méthode principale :
#   search_manuals(query, k=5, filter_expr="type eq 'manuel'")
#     -> renvoie les résultats bruts (SearchResult-like) retournés par Azure.
# -----------------------------------------------------------------------------

class AzureSearchFilteredRetriever:
    def __init__(self, endpoint: str, index_name: str, key: str, embed_fn):
        # Enregistre les paramètres de connexion / embedding
        self.endpoint = endpoint
        self.index_name = index_name
        self.embed_fn = embed_fn
        # Client natif Azure Search
        self.client = SearchClient(endpoint, index_name, AzureKeyCredential(key))

    def search_manuals(self, query: str, k: int = 5, filter_expr: str = "type eq 'manuel'"):
        # 1. embed la requête
        vec = self.embed_fn(query)
        # 2. construit l’appel selon SDK
        if VectorizedQuery is not None:
            # Nouveau SDK
            vq = VectorizedQuery(vector=vec, k=k, fields="embedding")
            results = self.client.search(
                search_text="*",  # pas de fulltext, vector only
                vector_queries=[vq],
                filter=filter_expr,
            )
        else:
            # Ancien SDK
            from azure.search.documents.models import Vector  # fallback
            v = Vector(value=vec, k=k, fields="embedding")
            results = self.client.search(
                search_text="*",
                vector=v,
                filter=filter_expr,
            )

        docs = []
        for r in results:
            # r est un dict-like
            docs.append(r)
        return docs


#### Création d'une classe pour le RAG

In [ ]:
# -----------------------------------------------------------------------------
# Configuration logging simple pour afficher les étapes du pipeline.
# -----------------------------------------------------------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)
logger = logging.getLogger(__name__)


# -----------------------------------------------------------------------------
# RAGPipelineAzure
# -----------------------------------------------------------------------------
# Ce pipeline combine :
#   - un modèle d'embedding HuggingFace (utilisé pour vectoriser les requêtes)
#   - un LLM hébergé sur Azure OpenAI (chat completions)
#   - un index Azure Cognitive Search contenant des documents vectorisés
#
# self.search_client : client natif Azure + retrieve_filtered() -> filtre OData
#
# Usage typique :
#   pipeline = RAGPipelineAzure(...config Terraform...)
#   réponse = pipeline.answer("Ma question", k=5)
#
# NB : l'index contient un champ 'type' (filterable=True) et
#       un champ vectoriel 'embedding' (dimensions cohérentes avec l'embedding_fn).
# -----------------------------------------------------------------------------

class RAGPipelineAzure:
    def __init__(
        self,
        model_name: str,
        api_key: str,
        api_base: str,
        api_version: str,
        deployment_name: str,
        config: Dict[str, Any],
        index_name: str = "documents",
    ) -> None:

        """
        Initialise la pipeline RAG avec Azure Cognitive Search vectoriel.
        """
        # Sauvegarde du nom du modèle d'embedding HuggingFace
        self.model_name = model_name
        # Initialise la fonction d'embedding (requêtes & docs)
        self.embedding_fn = HuggingFaceEmbeddings(model_name=model_name)
        # Client LLM Azure OpenAI (déploiement chat)
        self.llm = AzureChatOpenAI(
            azure_endpoint=api_base,
            api_key=api_key,
            api_version=api_version,
            deployment_name=deployment_name,
        )
        # Nom de l'index Search
        self.index_name = index_name

        # Client natif Search pour filtrage propre
        self.search_client = SearchClient(
            config["search_endpoint"],
            index_name,
            AzureKeyCredential(config["search_key"])
        )

        # Petite lambda utilitaire : embed une requête texte -> vecteur (liste de floats)
        self._embed_query = lambda txt: self.embedding_fn.embed_query(txt)

    def retrieve_filtered(
        self,
        query: str,
        k: int = 5,
        doc_type: Optional[str] = None
    ) -> List[Document]:

        """Vector search côté Azure + filtre OData sur type='...'."""
        # Embed de la requête utilisateur
        vec = self._embed_query(query)
        # Expression de filtre OData (doit matcher un champ filterable dans l'index)
        filter_expr = f"type eq '{doc_type}'" if doc_type else None

        results_iter = None
        try:
            # Tentative nouvelle API
            from azure.search.documents.models import VectorizedQuery
            vq = VectorizedQuery(vector=vec, k=k, fields="embedding")
            results_iter = self.search_client.search(
                search_text="*",  # vector-only pattern
                vector_queries=[vq],
                filter=filter_expr,
            )
        except ImportError:
            # Ancienne API
            from azure.search.documents.models import Vector
            v = Vector(value=vec, k=k, fields="embedding")
            results_iter = self.search_client.search(
                search_text="*",
                vector=v,
                filter=filter_expr,
            )
        except Exception as e:
            # Toute autre erreur (réseau, auth, schéma, etc.)
            logger.error(f"Recherche Azure Search échouée: {e}")
            raise

        # Conversion vers objets LangChain Document pour réutiliser la suite du pipeline
        from langchain.schema import Document
        docs = []
        for r in results_iter:
            # r est SearchResult -> accès dict-like
            docs.append(
                Document(
                    page_content=r.get("content", "") if hasattr(r, "get") else r["content"],
                    metadata={
                        "id": r["id"] if "id" in r else getattr(r, "id", None),
                        "filename": r.get("filename") if hasattr(r, "get") else r.get("filename", None) if isinstance(r, dict) else None,
                        "type": r.get("type") if hasattr(r, "get") else r.get("type", None) if isinstance(r, dict) else None,
                        "created": r.get("created") if hasattr(r, "get") else r.get("created", None) if isinstance(r, dict) else None,
                        "document_id": r.get("document_id") if hasattr(r, "get") else r.get("document_id", None) if isinstance(r, dict) else None,
                    },
                )
            )
        # On ne retourne que les k premiers (Azure en a déjà retourné k, mais on sécurise)
        return docs[:k]

    def answer(self, query: str, k: int = 5) -> str:
        """Renvoie une réponse générée par le LLM sur la base des docs filtrés."""
        try:
            # Récupération des meilleurs chunks filtrés côté Azure (type='manuel')
            results = self.retrieve_filtered(query, k=k, doc_type="manuel")
            # Concaténation du contenu des chunks pour former le contexte
            context = "\n\n".join([doc.page_content for doc in results])
            # Prompt instructif (contexte + question)
            prompt = f"""
                Tu es un assistant expert, spécialisé dans la documentation technique des vélos.

                Réponds de façon précise et concise à la question utilisateur suivante,
                en t’appuyant STRICTEMENT et EXCLUSIVEMENT sur les extraits ci-dessous (issus de la documentation technique officielle) :
                \"\"\"
                {context}
                \"\"\"

                INSTRUCTIONS :
                - Ne donne aucune information qui ne provient pas explicitement de ces extraits.
                - Si la réponse exacte (par exemple, une valeur numérique, un modèle, un tableau) est présente, cite-la textuellement et indique d'où elle provient.
                - Si plusieurs passages sont nécessaires pour une réponse complète, combine-les de façon claire.
                - Si l’information demandée n’apparaît pas dans les extraits, dis simplement : « L’information ne figure pas dans la documentation fournie. »
                - Reste factuel, sans extrapolation ni interprétation.

                Question :
                {query}
                """
            # Appel LLM Azure OpenAI
            logger.info("Envoi du prompt au LLM Azure OpenAI...")
            response = self.llm.invoke(prompt)
            logger.info("Retour LLM récupéré.")
            content = getattr(response, "content", None)
            if not content:
                logger.warning("Pas de contenu retourné par le LLM ou réponse vide.")
                return "Le LLM n'a rien répondu (réponse vide, sans contenu)"
            return content
        except Exception as e:
            logger.error(f"Erreur lors de la génération de la réponse LLM : {e}")
            return "Une erreur s'est produite lors de la génération de la réponse."


In [ ]:
pipeline = RAGPipelineAzure(
    model_name="sentence-transformers/all-MiniLM-L6-v2",  # Le modèle d'embedding HuggingFace utilisé à l'indexation
    api_key=config["openai_key"],                         # Clé API Azure OpenAI sortie du terraform
    api_base=config["openai_endpoint"],                   # Endpoint Azure OpenAI sorti du terraform
    api_version="2024-02-15-preview",                     # Version API Azure OpenAI (à ajouter dans ta classe, cf plus bas)
    deployment_name=config["chat_deployment"],            # Nom du déploiement Azure OpenAI
    config=config,
    index_name="documents"
)

### Tests simple pour vérifier le résultat

In [ ]:
# Réponse
print(pipeline.answer("En quelles couleurs est disponible le modèle urbain classic ?"))

In [ ]:
print(pipeline.answer("Quel est le poids du Urbain-Confort ?"))

In [ ]:
print(pipeline.answer("Que faire si la chaîne du vélo Pro-Livraison saute ?"))

### Test pour une information absente des manuels

In [ ]:
print(pipeline.answer("Quel est le diamètre des roues du vélo Pro-Livraison ?"))

# 2/ RAG Agentique simple
### -> 2 sources de données : bases SQL et documentation technique (manuels)
- L'orchestrateur choisit quels tools utiliser et renvoie une réponse

In [ ]:
# Embedding model (chargé une seule fois)
def _get_embedder(model_name: str = "sentence-transformers/all-MiniLM-L6-v2"):
    """Charge une seule fois le modèle d'embedding HuggingFace."""
    global _EMBED_MODEL
    if _EMBED_MODEL is None:
        from sentence_transformers import SentenceTransformer
        _EMBED_MODEL = SentenceTransformer(model_name)
    return _EMBED_MODEL

Première étape : définir les outils utilisables

In [ ]:
# DEBUG helpers : simple préfixe pour repérer les traces
DBG = lambda msg: print(f"[DBG] {msg}")

# Définition des Tools :

# La fonction suivante permet d'aller chercher via recherche sémantique un texte répondant à une question.
# Ce code est grandement inspiré de la classe précédente, mais sans génération de réponse directe
@tool
def search_documents(query: str, top_results: int = 3) -> str:
    """
    Recherche sémantique dans l’index Azure Cognitive Search **documents**.

    ⚙️  Fonctionnement
    ------------------
    1. Encode la requête utilisateur avec le modèle Sentence-Transformers
       déjà utilisé à l’indexation (cosine / dot product selon la config).
    2. Envoie une requête vectorielle (API `VectorizedQuery` ou `Vector`)
       et récupère les *k* documents les plus proches.
    3. Formate la réponse en Markdown :
       **n°. filename** – aperçu 200 caractères – *type* – *score*.

    Paramètres
    ----------
    query : str
        Texte brut de la question / mot-clé à rechercher.
    top_results : int, default = 3
        Nombre maximum de documents à renvoyer.

    Retour
    ------
    str
        - Une liste Markdown des résultats.
        - “Aucun document…” si rien trouvé.
        - Message d’erreur clair en cas d’exception.
    """

    client = SearchClient(config["search_endpoint"], "documents",
                          AzureKeyCredential(config["search_key"]))
    vec = _EMBED_MODEL.encode(query).tolist()

    # Choix API Vector / VectorizedQuery selon SDK
    try:
        from azure.search.documents.models import VectorizedQuery
        res = client.search(search_text="*",
                            vector_queries=[VectorizedQuery(vector=vec,
                                                            k=top_results,
                                                            fields="embedding")])
    except ImportError:
        from azure.search.documents.models import Vector
        res = client.search(search_text="*",
                            vector=Vector(value=vec, k=top_results,
                                          fields="embedding"))
    docs = list(res)
    if not docs:
        return f"Aucun document pour « {query} »."

    out: List[str] = []
    for i, d in enumerate(docs, 1):
        snippet = (d.get("content", "")[:200] + "…").replace("\n", " ")
        out.append(f"**{i}. {d.get('filename','?')}** – {snippet}")
    return "\n".join(out)

# ---------------------------------------------------------------------------

# Ce tool permet d'explorer les tables SQL
@tool
def explore_database_schema(table_name: str = None) -> str:
    """
    Explore la base SQL Server VéloCorp :

    - Sans argument → liste toutes les tables.
    - Avec `table_name` → détail des colonnes de cette table.

    Paramètres
    ----------
    table_name : str | None
        Nom exact de la table (sensible à la casse selon la collation).

    Retour
    ------
    str
        - Markdown listant les tables OU
        - Tableau Markdown « colonne / type » pour la table ciblée
        - Message d’erreur lisible si la table n’existe pas.

    Bonnes pratiques
    ----------------
    - Appeler cette fonction **avant** de composer une requête SQL complexe.
    - Coupler avec `query_database` pour inspecter un échantillon.
    """
    conn = (f"DRIVER={{ODBC Driver 18 for SQL Server}};"
            f"SERVER={config['sql_server']};DATABASE={config['sql_database']};"
            f"UID={config['sql_username']};PWD={config['sql_password']};"
            "Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;")
    with pyodbc.connect(conn) as cnx:
        cur = cnx.cursor()
        if table_name is None:
            cur.execute("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES "
                        "WHERE TABLE_TYPE='BASE TABLE' ORDER BY 1")
            tables = "\n".join("- " + r[0] for r in cur.fetchall())
            DBG("Liste des tables SQL :\n" + tables)
            return tables

        cur.execute("SELECT COLUMN_NAME, DATA_TYPE "
                    "FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME=?",
                    table_name)
        cols = cur.fetchall()
        if not cols:
            DBG(f"Table introuvable : {table_name}")
            return f"Table « {table_name} » introuvable."
        detail = f"**{table_name}**\n" + "\n".join(f"- {c[0]} ({c[1]})"
                                                   for c in cols)
        DBG(f"Schéma de {table_name} :\n" + detail)
        return detail

# ---------------------------------------------------------------------------

# Ce Tool permet de faire une requête au serveur SQL
@tool
def query_database(sql_query: str) -> str:
    """
    Exécute une requête SELECT uniquement sur SQL Server.

    Sécurité : toute commande non-`SELECT` est rejetée immédiatement.

    Paramètres
    ----------
    sql_query : str
        Requête SQL complète (peut inclure JOIN, CTE, OFFSET/FETCH, etc.).

    Retour
    ------
    str
        - Tableau Markdown (max 10 lignes)
        - “Aucun résultat.” si la sélection est vide
        - Message d’erreur enrichi (syntaxe, table inconnue, permissions…).

    Exemple
    -------
    ```python
    query_database(
        \"\"\"SELECT TOP 5 customer_id, SUM(total) AS CA
            FROM invoices GROUP BY customer_id ORDER BY CA DESC\"\"\")
    ```
    """
    # 1) Sécurité basique : on bloque tout ce qui n’est pas SELECT
    if not sql_query.strip().upper().startswith("SELECT"):
        print("[DBG] Blocage sécurité : non-SELECT")
        return "Seules les requêtes SELECT sont autorisées."

    # 2) Connexion SQL Server
    conn_str = (f"DRIVER={{ODBC Driver 18 for SQL Server}};"
                f"SERVER={config['sql_server']};DATABASE={config['sql_database']};"
                f"UID={config['sql_username']};PWD={config['sql_password']};"
                "Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;")

    print("[DBG] SQL envoyé :")
    print(sql_query)

    with pyodbc.connect(conn_str) as conn:
        cur = conn.cursor()
        cur.execute(sql_query)

        cols = [d[0] for d in cur.description]
        rows = cur.fetchmany(max_rows)        # ⚠️ on charge max_rows, pas tout

    # 3) Debug : combien de lignes ont été renvoyées ?
    print(f"[DBG] Lignes ramenées : {len(rows)} / limite affichage {max_rows}")

    if not rows:
        print("[DBG] Colonnes reçues :", cols)
        return "Aucun résultat."

    # 4) Formatage Markdown (identique à ta version)
    header = " | ".join(cols)
    sep    = "-" * len(header)
    body   = [" | ".join(str(c) for c in r) for r in rows]
    return "```\n" + "\n".join([header, sep, *body]) + "\n```"

Deuxième étape : contruiure l'agent LangGraph

In [ ]:
# Construction de l'agent LangGraph

# LLM (Azure OpenAI ou OpenAI “classique”)
llm = AzureChatOpenAI(
    azure_endpoint  = config["openai_endpoint"],
    api_key         = config["openai_key"],
    deployment_name = config["chat_deployment"],
    api_version     = "2024-02-15-preview"
)

# Bind des outils → le LLM générera directement les appels
llm_tools = llm.bind_tools([search_documents,
                            explore_database_schema,
                            query_database])

# État pour LangGraph
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]

# -------------------------------------------------------------------
def agent_node(state: AgentState):
    system = """
    Tu es **VéloCorpGPT**, assistant technique & data.

    Outils disponibles
    ------------------
    1. `search_documents` : recherche sémantique dans la doc interne (manuel, FAQ…).
    2. `explore_database_schema` : inspecte la structure SQL (tables / colonnes).
    3. `query_database` : exécute des requêtes SELECT (10 lignes max).

    Stratégie recommandée
    ---------------------
    - Pour des questions métier (ventes, clients, stock) :
      1. Commence par `explore_database_schema()` si tu n’es pas certain de la table.
      2. Rédige UNE requête SQL complète, puis `query_database()`.
      3. Reformule la réponse pour l’utilisateur (unités, sommes, ordres de grandeur).

    - Pour de la documentation technique ou produit :
      1. `search_documents()` avec des mots-clés précis.
      2. Synthétise la réponse à partir des extraits retournés.

    Règles
    ------
    - Utilise autant d’outils que nécessaire avant de répondre.
    - Ne devine jamais un schéma SQL : vérifie-le.
    - Garde un style concis et structuré (titres `###`, listes à puces, tableaux).

    Autres règles relatives aux requêtes SQL:
    1. **Un SEUL SQL principal** pour répondre : construis une requête agrégée complète (JOIN/CTE si besoin) au lieu de plusieurs petites requêtes indépendantes.
    2. **Filtre de dates** : utilise des intervalles (`date >= 'YYYY-01-01' AND date < 'YYYY-02-01'`) au lieu de `MONTH()`/`YEAR()` (meilleure perf + index friendly).
    3. **Auto-contrôle après exécution** :
      - Si un champ agrégé ressort `NULL`, REFORMULE/REJOINS pour renvoyer 0 au lieu de NULL.
      - Si le résultat paraît incohérent (ex: 10 commandes mais total NULL), relance une requête corrigée.
    4. **Toujours afficher la requête exécutée** et le sens de chaque colonne retournée dans la réponse finale.
    5. **Agrégations sûres** : enveloppe toute `SUM(...)` ou `COUNT(...)` susceptibles d’être vides dans `COALESCE(...,0)` pour éviter `NULL`.
    §. SURTOUT ne devine jamais une table ou une colonne: vérifie
    """
    msgs   = [HumanMessage(content=system)] + state["messages"]
    resp   = llm_tools.invoke(msgs)
    return {"messages": state["messages"] + [resp]}

def should_continue(state: AgentState):
    """
    Décide si le workflow LangGraph doit :

    - exécuter les appels d’outils renvoyés par le LLM (“tools”)
    - ou s’arrêter (END) et retourner la réponse finale à l’utilisateur.

    Règle :
    --------
    Si le dernier message contient la clé `tool_calls` (c.-à-d. que le LLM
    a demandé un ou plusieurs outils), on branche vers le nœud « tools ».
    Sinon, on considère que la réponse est complète et on termine.
    """
    last = state["messages"][-1]
    if getattr(last, "tool_calls", None):
        return "tools"
    return END

graph = StateGraph(AgentState)
graph.add_node("agent", agent_node)
graph.add_node("tools", ToolNode([search_documents,
                                  explore_database_schema,
                                  query_database]))
graph.set_entry_point("agent")
graph.add_conditional_edges("agent", should_continue,
                            {"tools": "tools", END: END})
graph.add_edge("tools", "agent")
rag_agent = graph.compile()


Dernière étape : création d'une fonction pour dialoguer

In [ ]:
def ask(question: str):
    out = rag_agent.invoke({"messages": [HumanMessage(content=question)]})
    return out["messages"][-1].content

Exemples d'utilisations :

In [ ]:
# Exemple 1: Requête dans la base de données
ask("Combien de commandes avons-nous eu au mois de juillet et quelle est leur valeur totale?")

# Point à noter: il arrive que parfois l'agent n'arrive pas à calculer le montant et renvoie zéro euros
# N'hésitez pas à relancer pour obtenir le résultat.
# On touche ici aux limites d'un agent simple avec des tools limités
# => La requête SQL formulée par le LLM n'est pas toujours correcte.
# Typiquement pour cet exemple un workflow serait plus adapté
# (la logique de sortir dans un premier temps le schéma et de bâtir la requête étant systématique)

## 🛠️ Tool tracker

In [ ]:
# Classe pour tracker l'usage des outils
class ToolUsageTracker:
    def __init__(self):
        self.tool_calls = []
        self.session_start = datetime.now()

    def track_tool_call(self, tool_name: str, args: dict, start_time: float, end_time: float,
                       success: bool, result_summary: str = "", error: str = ""):
        """Enregistre l'utilisation d'un outil"""
        self.tool_calls.append({
            "tool_name": tool_name,
            "args": args,
            "start_time": datetime.fromtimestamp(start_time),
            "end_time": datetime.fromtimestamp(end_time),
            "duration_ms": round((end_time - start_time) * 1000, 2),
            "success": success,
            "result_summary": result_summary,
            "error": error
        })

    def get_usage_summary(self):
        """Retourne un résumé de l'usage des outils"""
        if not self.tool_calls:
            return "Aucun outil utilisé"

        summary = {
            "total_calls": len(self.tool_calls),
            "total_duration_ms": sum(call["duration_ms"] for call in self.tool_calls),
            "success_rate": len([c for c in self.tool_calls if c["success"]]) / len(self.tool_calls) * 100,
            "tools_used": list(set(call["tool_name"] for call in self.tool_calls)),
            "calls_by_tool": {}
        }

        for call in self.tool_calls:
            tool = call["tool_name"]
            if tool not in summary["calls_by_tool"]:
                summary["calls_by_tool"][tool] = {
                    "count": 0,
                    "total_duration_ms": 0,
                    "success_count": 0
                }

            summary["calls_by_tool"][tool]["count"] += 1
            summary["calls_by_tool"][tool]["total_duration_ms"] += call["duration_ms"]
            if call["success"]:
                summary["calls_by_tool"][tool]["success_count"] += 1

        return summary

    def get_detailed_log(self):
        """Retourne le log détaillé des appels d'outils"""
        return self.tool_calls

# Instance globale du tracker
tool_tracker = ToolUsageTracker()

# Wrapper pour tracker les outils automatiquement
def tracked_tool(func):
    """Décorateur pour tracker automatiquement l'usage des outils"""

    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        tool_name = func.__name__
        start_time = time.time()

        try:
            result = func(*args, **kwargs)
            end_time = time.time()

            # Résumé du résultat (premiers 100 caractères)
            result_summary = str(result)[:100] + "..." if len(str(result)) > 100 else str(result)

            tool_tracker.track_tool_call(
                tool_name=tool_name,
                args={"args": args, "kwargs": kwargs},
                start_time=start_time,
                end_time=end_time,
                success=True,
                result_summary=result_summary
            )

            return result

        except Exception as e:
            end_time = time.time()

            tool_tracker.track_tool_call(
                tool_name=tool_name,
                args={"args": args, "kwargs": kwargs},
                start_time=start_time,
                end_time=end_time,
                success=False,
                error=str(e)
            )

            raise e

    return wrapper


## 🔧 Configuration des Sources de Données

In [ ]:

try:
    # Nouvelle API vectorielle
    from azure.search.documents.models import VectorizedQuery
    _HAS_VQ = True
except ImportError:
    _HAS_VQ = False

# Cache global de l'embedder HuggingFace pour éviter de recharger à chaque appel
_EMBED_MODEL = None

def _get_embedder(model_name: str = "sentence-transformers/all-MiniLM-L6-v2"):
    """Charge une seule fois le modèle d'embedding HuggingFace."""
    global _EMBED_MODEL
    if _EMBED_MODEL is None:
        from sentence_transformers import SentenceTransformer
        _EMBED_MODEL = SentenceTransformer(model_name)
    return _EMBED_MODEL

# Outils avec tracking automatique
@tool
@tracked_tool
def search_documents(query: str, top_results: int = 3) -> str:
    """Recherche vectorielle (similarité cosinus) dans l'index Azure Search 'documents'.

    On encode la requête avec le même modèle que celui utilisé pour indexer les embeddings,
    puis on envoie une requête vectorielle à Azure Search. Résultats triés par proximité
    (cosine / dot, selon la config de l'index HNSW).

    Args:
        query: Texte de la requête utilisateur.
        top_results: Nombre maximum de résultats à retourner.

    Returns:
        Chaîne formatée en Markdown listant les meilleurs documents.
    """
    try:
        # --- Connexion Azure Search --------------------------------------------------
        search_credential = AzureKeyCredential(config["search_key"])
        search_client = SearchClient(config["search_endpoint"], "documents", search_credential)

        # --- Embedding de la requête -------------------------------------------------
        embedder = _get_embedder()  # charge/cashe le modèle HF
        query_vec = embedder.encode(query)  # -> numpy array
        # Azure attend une liste de floats (pas un np.ndarray)
        query_vec = query_vec.tolist()

        # --- Construction de la requête vectorielle ---------------------------------
        if _HAS_VQ:
            # Nouvelle API (>= 11.5 environ)
            vq = VectorizedQuery(vector=query_vec, k=top_results, fields="embedding")
            results_iter = search_client.search(
                search_text="*",               # vector-only pattern
                vector_queries=[vq],
            )
        else:
            # Ancienne API (Vector)
            from azure.search.documents.models import Vector
            v = Vector(value=query_vec, k=top_results, fields="embedding")
            results_iter = search_client.search(
                search_text="*",
                vector=v,
            )

        # --- Collecte & formatage ----------------------------------------------------
        results = list(results_iter)
        if not results:
            return f"Aucun document trouvé (vector search) pour '{query}'."

        formatted_results = []
        for i, result in enumerate(results, 1):
            # result est SearchResult dict-like
            filename = result.get("filename", "N/A")
            doc_type = result.get("type", "N/A")
            content_preview = (result.get("content", "") or "")[:200] + "..."
            score = result.get("@search.score", 0)

            formatted_results.append(
                f"**Document {i}: {filename}** *(type: {doc_type}, score: {score:.4f})*\n"
                f"{content_preview}\n"
            )

        return "\n".join(formatted_results)

    except Exception as e:
        return f"Erreur recherche vectorielle documents: {str(e)}"


@tool
@tracked_tool
def query_database(sql_query: str) -> str:
    """Exécute une requête SQL sur la base de données VéloCorp.

    Args:
        sql_query: Requête SQL à exécuter (SELECT uniquement)

    Returns:
        Résultats de la requête formatés

    Note: Si la requête échoue à cause d'une table/colonne inconnue,
    utilise explore_database_schema() pour voir la structure.
    """
    # Vérification de sécurité - uniquement SELECT
    if not sql_query.strip().upper().startswith('SELECT'):
        return "❌ Erreur: Seules les requêtes SELECT sont autorisées"

    try:
        connection_string = f"DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={config['sql_server']};DATABASE={config['sql_database']};UID={config['sql_username']};PWD={config['sql_password']};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"

        with pyodbc.connect(connection_string) as conn:
            cursor = conn.cursor()
            cursor.execute(sql_query)

            # Récupération des colonnes
            columns = [desc[0] for desc in cursor.description]
            rows = cursor.fetchall()

            if not rows:
                return "ℹ️ Aucun résultat trouvé pour cette requête"

            # Formatage des résultats
            result_lines = [" | ".join(columns)]
            result_lines.append("-" * len(result_lines[0]))

            for row in rows[:10]:  # Limite à 10 résultats
                formatted_row = []
                for val in row:
                    if val is None:
                        formatted_row.append("NULL")
                    else:
                        formatted_row.append(str(val)[:50])  # Limite à 50 chars
                result_lines.append(" | ".join(formatted_row))

            if len(rows) > 10:
                result_lines.append(f"... et {len(rows) - 10} autres résultats")

            # Ajout d'informations contextuelles
            result = "\n".join(result_lines)
            result += f"\n\n✅ **{len(rows)} résultat(s) trouvé(s)**"

            return result

    except Exception as e:
        error_msg = str(e).lower()

        # Messages d'erreur intelligents avec suggestions
        if "invalid object name" in error_msg or "invalid column name" in error_msg:
            suggestion = "\n\n💡 **Suggestion**: La table ou colonne semble inexistante. "
            suggestion += "Utilise explore_database_schema() pour voir les tables disponibles, "
            suggestion += "ou explore_database_schema('nom_table') pour voir la structure d'une table."
            return f"❌ Erreur SQL: {str(e)}{suggestion}"

        elif "syntax error" in error_msg:
            suggestion = "\n\n💡 **Suggestion**: Erreur de syntaxe SQL. "
            suggestion += "Utilise get_sql_examples() pour voir des exemples de requêtes."
            return f"❌ Erreur de syntaxe: {str(e)}{suggestion}"

        elif "permission" in error_msg or "access" in error_msg:
            return f"❌ Erreur de permissions: {str(e)}\nSeules les requêtes SELECT sont autorisées."

        else:
            return f"❌ Erreur base de données: {str(e)}"

@tool
@tracked_tool
def query_crm_api(endpoint: str, params: dict = None) -> str:
    """Interroge l'API CRM VéloCorp.

    Args:
        endpoint: Endpoint à appeler (commerciaux, prospects, opportunites, analytics)
        params: Paramètres optionnels de la requête

    Returns:
        Réponse de l'API formatée
    """
    try:
        url = f"{config['api_base_url']}/crm/{endpoint}"

        response = requests.get(url, params=params or {}, timeout=30)
        response.raise_for_status()

        data = response.json()

        # Formatage selon l'endpoint
        if endpoint == "commerciaux":
            commerciaux = data.get('commerciaux', [])
            result = f"**{data.get('count', 0)} commerciaux trouvés:**\n"
            for com in commerciaux[:5]:
                result += f"- {com['name']} ({com['email']}) - Régions: {', '.join(com['assigned_regions'])}\n"
            return result

        elif endpoint == "prospects":
            prospects = data.get('prospects', [])
            resume = data.get('resume', {})
            result = f"**{data.get('count', 0)} prospects trouvés:**\n"
            result += f"Score moyen: {resume.get('score_moyen', 0)}\n"
            for prospect in prospects[:3]:
                result += f"- {prospect['contact_name']} ({prospect['company']}) - Score: {prospect['lead_score']}\n"
            return result

        elif endpoint == "opportunites":
            opportunites = data.get('opportunites', [])
            metriques = data.get('metriques_pipeline', {})
            result = f"**{data.get('count', 0)} opportunités trouvées:**\n"
            result += f"Valeur pipeline: {metriques.get('valeur_pipeline', 0):,.2f}€\n"
            for opp in opportunites[:3]:
                result += f"- {opp['title']} - {opp['estimated_value']:,.2f}€ ({opp['status']})\n"
            return result

        elif endpoint == "analytics":
            globales = data.get('metriques_globales', {})
            result = "**Analytics CRM:**\n"
            result += f"Total prospects: {globales.get('total_prospects', 0)}\n"
            result += f"Total opportunités: {globales.get('total_opportunites', 0)}\n"
            result += f"Valeur pipeline: {globales.get('valeur_pipeline', 0):,.2f}€\n"
            result += f"Valeur gagnée: {globales.get('valeur_gagnee', 0):,.2f}€\n"
            return result

        else:
            return json.dumps(data, indent=2, ensure_ascii=False)[:500] + "..."

    except Exception as e:
        return f"Erreur API CRM: {str(e)}"



In [ ]:
@tool
@tracked_tool
def explore_database_schema(table_name: str = None) -> str:
    """Explore le schéma de la base de données VéloCorp.

    Args:
        table_name: Nom de la table à explorer (optionnel, si vide retourne toutes les tables)

    Returns:
        Structure des tables et colonnes
    """
    try:
        connection_string = f"DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={config['sql_server']};DATABASE={config['sql_database']};UID={config['sql_username']};PWD={config['sql_password']};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"

        with pyodbc.connect(connection_string) as conn:
            cursor = conn.cursor()

            if table_name:
                # Exploration d'une table spécifique
                query = """
                SELECT
                    c.COLUMN_NAME,
                    c.DATA_TYPE,
                    c.IS_NULLABLE,
                    c.COLUMN_DEFAULT,
                    CASE WHEN pk.COLUMN_NAME IS NOT NULL THEN 'PK' ELSE '' END as IS_PRIMARY_KEY,
                    CASE WHEN fk.COLUMN_NAME IS NOT NULL THEN
                        'FK -> ' + fk.REFERENCED_TABLE_NAME + '(' + fk.REFERENCED_COLUMN_NAME + ')'
                    ELSE '' END as FOREIGN_KEY
                FROM INFORMATION_SCHEMA.COLUMNS c
                LEFT JOIN (
                    SELECT ku.TABLE_NAME, ku.COLUMN_NAME
                    FROM INFORMATION_SCHEMA.TABLE_CONSTRAINTS tc
                    JOIN INFORMATION_SCHEMA.KEY_COLUMN_USAGE ku
                        ON tc.CONSTRAINT_NAME = ku.CONSTRAINT_NAME
                    WHERE tc.CONSTRAINT_TYPE = 'PRIMARY KEY'
                ) pk ON c.TABLE_NAME = pk.TABLE_NAME AND c.COLUMN_NAME = pk.COLUMN_NAME
                LEFT JOIN (
                    SELECT
                        ku.TABLE_NAME, ku.COLUMN_NAME,
                        ku2.TABLE_NAME as REFERENCED_TABLE_NAME,
                        ku2.COLUMN_NAME as REFERENCED_COLUMN_NAME
                    FROM INFORMATION_SCHEMA.REFERENTIAL_CONSTRAINTS rc
                    JOIN INFORMATION_SCHEMA.KEY_COLUMN_USAGE ku
                        ON rc.CONSTRAINT_NAME = ku.CONSTRAINT_NAME
                    JOIN INFORMATION_SCHEMA.KEY_COLUMN_USAGE ku2
                        ON rc.UNIQUE_CONSTRAINT_NAME = ku2.CONSTRAINT_NAME
                ) fk ON c.TABLE_NAME = fk.TABLE_NAME AND c.COLUMN_NAME = fk.COLUMN_NAME
                WHERE c.TABLE_NAME = ?
                ORDER BY c.ORDINAL_POSITION
                """
                cursor.execute(query, table_name)
                columns = cursor.fetchall()

                if not columns:
                    return f"Table '{table_name}' non trouvée"

                result = f"**Structure de la table {table_name}:**\n"
                result += "| Colonne | Type | Nullable | Défaut | Clé |\n"
                result += "|---------|------|----------|--------|-----|\n"

                for col in columns:
                    key_info = f"{col[4]} {col[5]}".strip()
                    result += f"| {col[0]} | {col[1]} | {col[2]} | {col[3] or 'NULL'} | {key_info} |\n"

                # Ajouter un échantillon de données
                sample_query = f"SELECT TOP 3 * FROM {table_name}"
                cursor.execute(sample_query)
                sample_data = cursor.fetchall()

                if sample_data:
                    result += f"\n**Échantillon de données:**\n"
                    column_names = [desc[0] for desc in cursor.description]
                    result += " | ".join(column_names) + "\n"
                    result += "-" * len(" | ".join(column_names)) + "\n"

                    for row in sample_data:
                        formatted_row = [str(val) if val is not None else 'NULL' for val in row]
                        result += " | ".join(formatted_row) + "\n"

                return result

            else:
                # Liste de toutes les tables
                query = """
                SELECT
                    t.TABLE_NAME,
                    COUNT(c.COLUMN_NAME) as COLUMN_COUNT,
                    STRING_AGG(c.COLUMN_NAME, ', ') as COLUMNS
                FROM INFORMATION_SCHEMA.TABLES t
                LEFT JOIN INFORMATION_SCHEMA.COLUMNS c ON t.TABLE_NAME = c.TABLE_NAME
                WHERE t.TABLE_TYPE = 'BASE TABLE'
                GROUP BY t.TABLE_NAME
                ORDER BY t.TABLE_NAME
                """
                cursor.execute(query)
                tables = cursor.fetchall()

                result = "**Tables disponibles dans la base VéloCorp:**\n"
                for table in tables:
                    result += f"- **{table[0]}** ({table[1]} colonnes)\n"
                    if table[2]:
                        columns_preview = table[2][:100] + "..." if len(table[2]) > 100 else table[2]
                        result += f"  Colonnes: {columns_preview}\n"

                result += "\nUtilise explore_database_schema('nom_table') pour plus de détails sur une table spécifique."
                return result

    except Exception as e:
        return f"Erreur exploration schéma: {str(e)}"

In [ ]:
# Liste des outils disponibles
tools = [search_documents, query_database, query_crm_api, explore_database_schema]

# LLM avec outils
llm_with_tools = llm.bind_tools(tools)

# État du graphe
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]

def agent_node(state: AgentState):
    """Nœud principal de l'agent avec instructions détaillées pour la DB"""
    system_prompt = """
    Tu es un assistant intelligent pour VéloCorp, entreprise de vélos.
    Tu as accès à 5 outils spécialisés :

    ## 🔍 OUTILS DISPONIBLES:

    1. **explore_database_schema()** - Explorer la structure de la DB
       - Sans paramètre : liste toutes les tables
       - Avec nom_table : détails d'une table spécifique

    2. **get_sql_examples(category)** - Exemples de requêtes SQL
       - Catégories : clients, commandes, produits, analytics, all

    3. **query_database(sql_query)** - Exécuter des requêtes SQL
       - Uniquement SELECT autorisé
       - Messages d'erreur intelligents avec suggestions

    4. **search_documents(query)** - Rechercher dans la documentation
       - Manuels, FAQ, emails internes

    5. **query_crm_api(endpoint)** - API CRM
       - Endpoints : commerciaux, prospects, opportunites, analytics

    ##### 🎯 STRATÉGIE OPTIMALE:

    ### STRATÉGIE OUTILS

    - Étape 1 : `explore_database_schema()` (global puis tables candidates).
    - Étape 2 : Génère la requête SQL complète → `query_database()`.
    - Étape 3 : Si résultat incomplet/NULL, corrige et relance (ne PAS répondre tant que les 2 métriques demandées ne sont pas numériques).

    Autres règles relatives aux requêtes SQL:
    1. **Un SEUL SQL principal** pour répondre : construis une requête agrégée complète (JOIN/CTE si besoin) au lieu de plusieurs petites requêtes indépendantes.
    2. **Toujours vérifier le schéma avant d’écrire du SQL** avec `explore_database_schema()` et résumer mentalement:
      - quelles tables contiennent les commandes ?
      - où se trouve le montant (orders.total ? order_items.unit_price*quantity ? invoices.amount ?).
    3. **Filtre de dates** : utilise des intervalles (`date >= 'YYYY-01-01' AND date < 'YYYY-02-01'`) au lieu de `MONTH()`/`YEAR()` (meilleure perf + index friendly).
    4. **Agrégations sûres** : enveloppe toute `SUM(...)` ou `COUNT(...)` susceptibles d’être vides dans `COALESCE(...,0)` pour éviter `NULL`.
    5. **Préserver le comptage des commandes même sans facture** : utilise des `LEFT JOIN` depuis `orders` vers les autres tables.
    6. **Auto-contrôle après exécution** :
      - Si un champ agrégé ressort `NULL`, REFORMULE/REJOINS pour renvoyer 0 au lieu de NULL.
      - Si le résultat paraît incohérent (ex: 10 commandes mais total NULL), relance une requête corrigée.
    7. **Toujours afficher la requête exécutée** et le sens de chaque colonne retournée dans la réponse finale.
    8. **Ne pas confondre “valeur des commandes” et “montant facturé”** :
      - si la question dit “valeur des commandes”, calcule à partir d’`orders` ou `order_items`; les factures ne sont qu’un proxy éventuel.

    Respecte ces règles avant de répondre.

    **Pour les questions sur la base de données :**
    1. 🔍 **TOUJOURS commencer par explore_database_schema()** si tu ne connais pas la structure exacte
    2. 📚 Si besoin d'inspiration pour les requêtes → get_sql_examples()
    3. ⚡ Puis exécuter → query_database()

    **Exemples de workflow :**
    - "Nos meilleurs clients" → explore_database_schema('clients') → query_database(...)
    - "Ventes du mois" → explore_database_schema('commandes') → query_database(...)
    - "Stock vélos" → explore_database_schema('produits') → query_database(...)

    **Pour autres types de questions :**
    - Documentation/FAQ → search_documents()
    - Performance commerciale → query_crm_api()

    ## ⚡ RÈGLES IMPORTANTES:
    - Ne devines JAMAIS la structure des tables
    - Utilise explore_database_schema() avant toute requête SQL complexe
    - Sois précis et concis dans tes réponses
    - Priorise la qualité des données sur la rapidité

    **Tu es maintenant prêt à aider efficacement avec VéloCorp !** 🚴‍♂️
    """

    '''
    messages = [HumanMessage(content=system_prompt)] + state["messages"]
    print("Prompt envoyé au LLM avec outils:", messages)
    response = llm_with_tools.invoke(messages)
    print("réponse brute du LLM")
    return {"messages": [response]}
    '''
    messages = [HumanMessage(content=system_prompt)] + state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": state["messages"] + [response]}


# Fonction de routage
def should_continue(state: AgentState):
    """Détermine si on continue ou on termine"""
    print("Check du should_continue, messages actuels :", state["messages"])

    last_message = state["messages"][-1]
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "tools"
    print("On termine l'agent.")

    return END

# Construction du graphe
workflow = StateGraph(AgentState)

# Ajout des nœuds
print("Ajout du noeud agent")
workflow.add_node("agent", agent_node)
print("Ajout du noeud tools")
workflow.add_node("tools", ToolNode(tools))

# Définition des edges
workflow.set_entry_point("agent")
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "tools": "tools",
        END: END
    }
)
workflow.add_edge("tools", "agent")

# Compilation du graphe
app = workflow.compile()
print("🤖 Agent LangGraph créé avec succès!")

## 🤖 Création de l'Agent LangGraph

## 🎯 Fonction d'Interaction Simple

In [ ]:
def ask_agent(
    question: str,
    verbose: bool = True,
    show_tool_details: bool = False,
    show_usage_stats: bool = False
):
    """
    Pose une question à l'agent et affiche la réponse avec options de diagnostic
    """
    print("\n❓ **Question:**", question)
    print("-" * 50)

    # Reset du tracker pour cette question
    global tool_tracker
    tool_tracker = ToolUsageTracker()

    try:
        print("[DEBUG] Début exécution agent...")
        start_time = time.time()

        # ENVOI DE LA QUESTION À L'AGENT
        result = app.invoke({"messages": [HumanMessage(content=question)]})

        end_time = time.time()
        print("[DEBUG] Résultat brut de l'agent :")
        print(result)

        # TEST CLÉ: Y A-T-IL BIEN UN CHAMP 'messages' ?
        if "messages" not in result:
            print("[DEBUG] Pas de clé 'messages' dans le résultat !")
            return

        print("[DEBUG] Messages retournés :")
        print(result["messages"])

        # On prend le dernier message
        last_message = result["messages"][-1]
        print("[DEBUG] Dernier message :")
        print(last_message)

        # REGARDE LE CHAMP content
        if hasattr(last_message, 'content'):
            final_response = last_message.content
        elif isinstance(last_message, dict) and "content" in last_message:
            final_response = last_message["content"]
        else:
            print("[DEBUG] Aucun champ 'content' trouvé dans le dernier message.")
            final_response = ""

        print(f"🤖 **Réponse:** {final_response}")

        # Affichage du temps total
        total_time = round((end_time - start_time) * 1000, 2)
        print(f"\n⏱️ **Temps total:** {total_time}ms")

        # Reste de la logique (outils/diagnostic)
        if verbose or show_tool_details or show_usage_stats:
            usage_summary = tool_tracker.get_usage_summary()

            if verbose and usage_summary != "Aucun outil utilisé":
                print(f"\n🔧 **Outils utilisés:** {', '.join(usage_summary['tools_used'])}")
                print(f"📊 **Nombre d'appels:** {usage_summary['total_calls']}")
                print(f"✅ **Taux de succès:** {usage_summary['success_rate']:.1f}%")

            if show_usage_stats and usage_summary != "Aucun outil utilisé":
                print(f"\n📈 **Statistiques détaillées:**")
                print(f"   - Temps total outils: {usage_summary['total_duration_ms']}ms")

                for tool_name, stats in usage_summary['calls_by_tool'].items():
                    success_rate = (stats['success_count'] / stats['count']) * 100
                    avg_time = stats['total_duration_ms'] / stats['count']
                    print(f"   - {tool_name}: {stats['count']} appels, {avg_time:.1f}ms moyen, {success_rate:.1f}% succès")

            if show_tool_details and usage_summary != "Aucun outil utilisé":
                print(f"\n🔍 **Détails des appels d'outils:**")
                for i, call in enumerate(tool_tracker.get_detailed_log(), 1):
                    status = "✅" if call['success'] else "❌"
                    print(f"   {i}. {status} {call['tool_name']} ({call['duration_ms']}ms)")
                    if call['args']['kwargs']:
                        print(f"      Args: {call['args']['kwargs']}")
                    if not call['success']:
                        print(f"      Erreur: {call['error']}")
                    elif call['result_summary']:
                        print(f"      Résultat: {call['result_summary']}")

    except Exception as e:
        print(f"❌ **Erreur:** {str(e)}")
        import traceback
        traceback.print_exc()

    print("\n" + "="*70)


### Illustration des retours du tool "explore_database_schema"

In [ ]:
print(explore_database_schema.invoke({}))


In [ ]:
print(explore_database_schema('customers'))

### Illustration du retour du Tool "query_api_crm"

In [ ]:
print(query_crm_api("commerciaux"))

# Exemples d'Interactions

In [ ]:
# Exemple 1: Requête dans la base de données
ask_agent("Combien de commandes avons-nous eu au mois de janvier et quelle est leur valeur totale?")

# Point à noter: il arrive que parfois l'agent n'arrive pas à calculer le montant et renvoie zéro euros
# N'hésitez pas à relancer pour obtenir le résultat.
# On touche ici aux limites d'un agent simple avec des tools limités
# => La requête SQL formulée par le LLM n'est pas toujours correcte.
# Typiquement pour cet exemple un workflow serait plus adapté
# (la logique de sortir dans un premier temps le schéma et de bâtir la requête étant systématique)

In [ ]:
# Exemple 2 : recherche vectorielle et CRM
ask_agent(
    question="Dans quelles couleurs est disponible le vélo urbain-confort et quels sont ses clients potentiels ?",
    verbose=True,
    show_tool_details=True,
    show_usage_stats=True)

In [ ]:
# Exemple 3: API CRM
ask_agent("Qui sont nos meilleurs clients et combien ont ils dépensé ?")

In [ ]:
# Exemple 3: API CRM
ask_agent(
    question="Qui sont nos meilleurs commerciaux et quelles sont leurs performances?",
    verbose=True,
    show_tool_details=True,
    show_usage_stats=True
)

In [ ]:
# Exemple 4: Question complexe nécessitant plusieurs sources
ask_agent("Quels sont nos produits les plus vendus et y a-t-il des opportunités commerciales en cours pour ces modèles?")

In [ ]:
# Exemple 4: Question complexe nécessitant plusieurs sources
ask_agent("Quels sont nos produits les plus vendus et y a-t-il des opportunités commerciales en cours pour ces modèles?")

In [ ]:
# Exemple 4: Question complexe nécessitant plusieurs sources
ask_agent("Quels sont nos produits les plus vendus et y a-t-il des opportunités commerciales en cours pour ces modèles?")

In [ ]:
# =======================================================================
#  Streamlit Chatbot VéloCorp – 100 % autonome dans UNE cellule
# =======================================================================
#
#  ⚠️  Prérequis (à installer une seule fois)
#  ------------------------------------------------
#  pip install streamlit langchain langgraph azure-search-documents \
#              sentence-transformers pyodbc requests
#
#  ⚙️  Variables d’environnement attendues
#  ------------------------------------------------
#  OPENAI_API_KEY          = <clé Azure OpenAI>
#  OPENAI_ENDPOINT         = <https://...>.openai.azure.com
#  OPENAI_DEPLOYMENT       = <nom du déploiement chat>
#  OPENAI_API_VERSION      = 2024-02-15-preview
#
#  SEARCH_ENDPOINT         = https://....search.windows.net
#  SEARCH_KEY              = <clé Admin ou Query>
#
#  SQL_SERVER              = <host.database.windows.net>
#  SQL_DATABASE            = <nom BDD>
#  SQL_USERNAME            = <login SQL>
#  SQL_PASSWORD            = <password>
#
#  API_BASE_URL            = https://api.velocorp.com
#
#  ➜ Adaptez si besoin, ou remplacez les os.getenv(...) ci-dessous par
#    des chaînes “en dur” pour un test rapide.
# -----------------------------------------------------------------------
import streamlit as st

# ----------------------------------------------------------------------
# 📋 Configuration (chargée depuis les variables d’environnement)
# ----------------------------------------------------------------------
config: Dict[str, str] = {
    "openai_key":        os.getenv("OPENAI_API_KEY"),
    "openai_endpoint":   os.getenv("OPENAI_ENDPOINT"),
    "chat_deployment":   os.getenv("OPENAI_DEPLOYMENT"),
    "openai_version":    os.getenv("OPENAI_API_VERSION", "2024-02-15-preview"),
    "search_endpoint":   os.getenv("SEARCH_ENDPOINT"),
    "search_key":        os.getenv("SEARCH_KEY"),
    "sql_server":        os.getenv("SQL_SERVER"),
    "sql_database":      os.getenv("SQL_DATABASE"),
    "sql_username":      os.getenv("SQL_USERNAME"),
    "sql_password":      os.getenv("SQL_PASSWORD"),
    "api_base_url":      os.getenv("API_BASE_URL"),
}

# ----------------------------------------------------------------------
# 🔧 Utilitaires & classes déjà fournis plus haut (inchangés)
#    ⬇️  Copiés/collés tels quels pour être self-contained
# ----------------------------------------------------------------------

logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s [%(levelname)s] %(message)s")
logger = logging.getLogger(__name__)

# ---------- Embeddings cache -------------------------------------------------
_EMBED_MODEL = None
def _get_embedder(model_name: str = "sentence-transformers/all-MiniLM-L6-v2"):
    global _EMBED_MODEL
    if _EMBED_MODEL is None:
        from sentence_transformers import SentenceTransformer
        _EMBED_MODEL = SentenceTransformer(model_name)
    return _EMBED_MODEL

# ---------- Tracker ----------------------------------------------------------
class ToolUsageTracker:
    def __init__(self): self.reset()
    def reset(self):
        self.tool_calls = []
        self.session_start = datetime.now()
    def track_tool_call(self, tool_name: str, args: dict,
                        start_time: float, end_time: float,
                        success: bool, result_summary: str = "", error: str = ""):
        self.tool_calls.append({
            "tool_name": tool_name,
            "args": args,
            "start_time": datetime.fromtimestamp(start_time),
            "end_time": datetime.fromtimestamp(end_time),
            "duration_ms": round((end_time - start_time) * 1000, 2),
            "success": success,
            "result_summary": result_summary,
            "error": error
        })
    def get_usage_summary(self):
        if not self.tool_calls: return "Aucun outil utilisé"
        s = {
            "total_calls": len(self.tool_calls),
            "total_duration_ms": sum(c["duration_ms"] for c in self.tool_calls),
            "tools_used": list({c["tool_name"] for c in self.tool_calls}),
        }
        return s
tool_tracker = ToolUsageTracker()

def tracked_tool(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start, name = time.time(), func.__name__
        try:
            res = func(*args, **kwargs)
            tool_tracker.track_tool_call(name, {"args": args, "kwargs": kwargs},
                                         start, time.time(), True,
                                         str(res)[:120])
            return res
        except Exception as e:
            tool_tracker.track_tool_call(name, {"args": args, "kwargs": kwargs},
                                         start, time.time(), False, error=str(e))
            raise
    return wrapper

# ---------- OUTILS LangChain -------------------------------------------------
@tool
@tracked_tool
def search_documents(query: str, top_results: int = 3) -> str:
    """Recherche vectorielle dans l’index Azure Cognitive Search 'documents'.
      Args:
          query: la requête texte de l’utilisateur.
          top_results: nombre de résultats max à retourner.
      Returns:
          Aperçu markdown des résultats ou message d’erreur.
    """

    try:
        client = SearchClient(config["search_endpoint"], "documents",
                              AzureKeyCredential(config["search_key"]))
        vec = _get_embedder().encode(query).tolist()
        if VectorizedQuery:
            vq = VectorizedQuery(vector=vec, k=top_results, fields="embedding")
            results = client.search(search_text="*", vector_queries=[vq])
        else:
            from azure.search.documents.models import Vector
            results = client.search(search_text="*",
                                    vector=Vector(value=vec, k=top_results,
                                                  fields="embedding"))
        out = []
        for i, r in enumerate(results, 1):
            out.append(f"**{i}. {r.get('filename','?')}** – " +
                       (r.get('content','')[:150].replace('\n',' ') + '…'))
        return "\n".join(out) or "Aucun résultat."
    except Exception as e:
        return f"Erreur : {e}"

@tool
@tracked_tool
def explore_database_schema(table_name: str = None) -> str:
    """Explore la structure de la base SQL VéloCorp."""
    try:
        conn_str = (
            f"DRIVER={{ODBC Driver 18 for SQL Server}};"
            f"SERVER={config['sql_server']};DATABASE={config['sql_database']};"
            f"UID={config['sql_username']};PWD={config['sql_password']};"
            "Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"
        )
        with pyodbc.connect(conn_str) as conn:
            cur = conn.cursor()
            if not table_name:
                cur.execute("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES "
                            "WHERE TABLE_TYPE='BASE TABLE' ORDER BY 1")
                return "\n".join("- " + r[0] for r in cur.fetchall())
            cur.execute("SELECT COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS "
                        "WHERE TABLE_NAME=?", table_name)
            cols = cur.fetchall()
            return f"**{table_name}**\n" + "\n".join(f"- {c[0]} ({c[1]})" for c in cols)
    except Exception as e:
        return f"Erreur schéma : {e}"

@tool
@tracked_tool
def query_database(sql_query: str) -> str:
    if not sql_query.strip().upper().startswith("SELECT"):
        """Exécute une requête SELECT sur la base VéloCorp et renvoie un tableau markdown."""
        return "Seules les requêtes SELECT sont autorisées."
    try:
        conn_str = (
            f"DRIVER={{ODBC Driver 18 for SQL Server}};"
            f"SERVER={config['sql_server']};DATABASE={config['sql_database']};"
            f"UID={config['sql_username']};PWD={config['sql_password']};"
            "Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"
        )
        with pyodbc.connect(conn_str) as conn:
            cur = conn.cursor()
            cur.execute(sql_query)
            cols = [d[0] for d in cur.description]
            rows = cur.fetchmany(20)
            header = " | ".join(cols)
            sep = "-" * len(header)
            lines = ["```\n" + header, sep]
            for r in rows:
                lines.append(" | ".join(str(x) for x in r))
            lines.append("```")
            return "\n".join(lines)
    except Exception as e:
        return f"Erreur SQL : {e}"

@tool
@tracked_tool
def query_crm_api(endpoint: str, params: dict = None) -> str:
    try:
        r = requests.get(f"{config['api_base_url']}/crm/{endpoint}",
                         params=params or {}, timeout=30)
        r.raise_for_status()
        return json.dumps(r.json()[:3], indent=2)[:500]
    except Exception as e:
        return f"Erreur API : {e}"

tools = [search_documents, explore_database_schema,
         query_database, query_crm_api]

# ---------- AGENT LangGraph --------------------------------------------------
llm = AzureChatOpenAI(
    azure_endpoint  = config["openai_endpoint"],
    api_key         = config["openai_key"],
    deployment_name = config["chat_deployment"],
    api_version     = config["openai_version"],
)

llm_tools = llm.bind_tools(tools)

class AgentState(TypedDict):
    messages: Annotated[list, add_messages]

def agent_node(state: AgentState):
    system_prompt = """
Tu es l’assistant VéloCorp. Utilise les outils si nécessaire.
Réponds de façon concise. Si besoin de SQL : commence par explore_database_schema().
"""
    msgs = [HumanMessage(content=system_prompt)] + state["messages"]
    resp = llm_tools.invoke(msgs)
    return {"messages": state["messages"] + [resp]}

def should_continue(state: AgentState):
    last = state["messages"][-1]
    if hasattr(last, "tool_calls") and last.tool_calls:
        return "tools"
    return END

graph = StateGraph(AgentState)
graph.add_node("agent", agent_node)
graph.add_node("tools", ToolNode(tools))
graph.set_entry_point("agent")
graph.add_conditional_edges("agent", should_continue,
                            {"tools": "tools", END: END})
graph.add_edge("tools", "agent")
app_graph = graph.compile()

def ask_agent(question: str) -> str:
    tool_tracker.reset()
    out = app_graph.invoke({"messages": [HumanMessage(content=question)]})
    return out["messages"][-1].content

# ----------------------------------------------------------------------
# 🌐  INTERFACE STREAMLIT (chat) – ultra-compact
# ----------------------------------------------------------------------
st.set_page_config(page_title="Chatbot VéloCorp", page_icon="🚴‍♂️",
                   layout="centered")
st.title("🤖 Chatbot VéloCorp")

if "history" not in st.session_state:
    st.session_state.history = []

for role, msg in st.session_state.history:
    st.chat_message(role).markdown(msg)

prompt = st.chat_input(placeholder="Posez votre question…")
if prompt:
    st.chat_message("user").markdown(prompt)
    with st.spinner("Réflexion…"):
        response = ask_agent(prompt)
    st.chat_message("assistant").markdown(response)
    st.session_state.history.append(("user", prompt))
    st.session_state.history.append(("assistant", response))

# ----------------------------------------------------------------------
# 🚀 Auto-lancement Streamlit (si la cellule est exécutée comme script)
# ----------------------------------------------------------------------
if __name__ == "__main__" and st._is_running_with_streamlit is False:
    # On écrit ce code dans un fichier temporaire et on lance `streamlit run`
    code = open(__file__, "r").read() if "__file__" in globals() else sys.modules[__name__].__dict__["In"][-1]
    with tempfile.NamedTemporaryFile("w", delete=False, suffix=".py") as f:
        f.write(code)
        script_path = f.name

    def _run():
        subprocess.call(["streamlit", "run", script_path, "--server.headless", "false"])

    threading.Thread(target=_run, daemon=True).start()
    time.sleep(3)
    webbrowser.open("http://localhost:8501")
